In [1]:
!pip install mysql-connector-python faker pandas streamlit

In [3]:
%%writefile config.py
DB_HOST = "localhost"  # Change if using a remote MySQL server
DB_USER = "root"
DB_PASSWORD = "vishaakh@2002"
DB_NAME = "student_placement"

Overwriting config.py


In [5]:
import mysql.connector
import config  # Import database credentials from config.py

try:
    connection = mysql.connector.connect(
        host=config.DB_HOST,
        user=config.DB_USER,
        password=config.DB_PASSWORD,
        database=config.DB_NAME
    )
    if connection.is_connected():
        print("Successfully connected to MySQL database")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if 'connection' in locals() and connection.is_connected():
        connection.close()

Successfully connected to MySQL database


In [7]:
%%writefile create_tables.py
import mysql.connector
import config  # Import database credentials from config.py

# Connect to MySQL
connection = mysql.connector.connect(
    host=config.DB_HOST,
    user=config.DB_USER,
    password=config.DB_PASSWORD,
    database=config.DB_NAME
)
cursor = connection.cursor()

# Creating tables
table_queries = {
    "Students": """
        CREATE TABLE IF NOT EXISTS Students (
            student_id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(100),
            age INT,
            gender ENUM('Male', 'Female', 'Other'),
            email VARCHAR(255) UNIQUE,
            phone VARCHAR(10) UNIQUE,
            enrollment_year INT,
            course_batch VARCHAR(20),
            city VARCHAR(100),
             graduation_year INT
        )
    """,
    
    "Programming": """
        CREATE TABLE IF NOT EXISTS Programming (
            programming_id INT AUTO_INCREMENT PRIMARY KEY,
            student_id INT,
            language VARCHAR(50),
            problems_solved INT,
            assessments_completed INT,
            mini_projects INT,
            certifications_earned INT,
            latest_project_score INT,
            FOREIGN KEY (student_id) REFERENCES Students(student_id) ON DELETE CASCADE
        )
    """,
    
    "SoftSkills": """
        CREATE TABLE IF NOT EXISTS SoftSkills (
            soft_skill_id INT AUTO_INCREMENT PRIMARY KEY,
            student_id INT,
            communication INT CHECK(communication BETWEEN 0 AND 100),
            teamwork INT CHECK(teamwork BETWEEN 0 AND 100),
            presentation INT CHECK(presentation BETWEEN 0 AND 100),
            leadership INT CHECK(leadership BETWEEN 0 AND 100),
             critical_thinking INT CHECK(critical_thinking BETWEEN 0 AND 100),
            interpersonal_skills INT CHECK(interpersonal_skills BETWEEN 0 AND 100),
            FOREIGN KEY (student_id) REFERENCES Students(student_id) ON DELETE CASCADE
        )
    """,
    
    "Placements": """
        CREATE TABLE IF NOT EXISTS Placements (
            placement_id INT AUTO_INCREMENT PRIMARY KEY,
            student_id INT,
            mock_interview_score INT CHECK(mock_interview_score BETWEEN 0 AND 100),
            internships_completed INT,
            placement_status ENUM('Ready', 'Not Ready', 'Placed'),
            company_name VARCHAR(255),
            placement_package DECIMAL(10,2),
            interview_rounds_cleared INT,
            placement_date DATE,
            FOREIGN KEY (student_id) REFERENCES Students(student_id) ON DELETE CASCADE
        )
    """
}

# Execute queries
for table_name, query in table_queries.items():
    cursor.execute(query)
    # Commit and close
connection.commit()
cursor.close()
connection.close()

print("Tables created successfully!")
    

Overwriting create_tables.py


In [9]:
!python create_tables.py

Tables created successfully!


In [11]:
import mysql.connector
import pandas as pd
import config  # Import database credentials

# Connect to MySQL
connection = mysql.connector.connect(
    host=config.DB_HOST,
    user=config.DB_USER,
    password=config.DB_PASSWORD,
    database=config.DB_NAME
)
cursor = connection.cursor()

# Function to fetch and display data
def fetch_data(query):
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])
    return df

# Sample queries
print(" First 5 students:")
display(fetch_data("SELECT * FROM Students LIMIT 5"))

print("\n First 5 programming records:")
display(fetch_data("SELECT * FROM Programming LIMIT 5"))
print("\n First 5 soft skills records:")
display(fetch_data("SELECT * FROM SoftSkills LIMIT 5"))

print("\n First 5 placement records:")
display(fetch_data("SELECT * FROM Placements LIMIT 5"))

# Close connection
cursor.close()
connection.close()

 First 5 students:


,student_id,name,age,gender,email,phone,enrollment_year,course_batch,city,graduation_year
0,4001,Lisa Bryant,23,Female,vanessa96@example.com,5920589884,2024,Batch 2,West Joshuamouth,2026
1,4002,David White,21,Other,christopher84@example.net,9535051429,2019,Batch 4,Toddshire,2023
2,4003,Amy Graham,18,Female,jonathanbeasley@example.org,8088307638,2024,Batch 8,New Manuel,2024
3,4004,Walter Norris,24,Male,mhenry@example.com,1497314334,2020,Batch 4,Fieldsshire,2023
4,4005,Jessica Potter,24,Male,ashleysmith@example.org,1546328441,2021,Batch 8,Lake Donaldfort,2025



 First 5 programming records:


,programming_id,student_id,language,problems_solved,assessments_completed,mini_projects,certifications_earned,latest_project_score
0,1,4001,SQL,38,5,3,2,79
1,2,4002,Python,148,5,5,3,52
2,3,4003,SQL,43,5,4,1,87
3,4,4004,SQL,171,3,2,3,95
4,5,4005,C++,79,4,2,0,88



 First 5 soft skills records:


,soft_skill_id,student_id,communication,teamwork,presentation,leadership,critical_thinking,interpersonal_skills
0,1,4001,57,65,62,84,90,61
1,2,4002,56,57,96,53,62,70
2,3,4003,99,86,97,56,75,90
3,4,4004,74,82,58,65,98,88
4,5,4005,97,96,88,67,72,59



 First 5 placement records:


,placement_id,student_id,mock_interview_score,internships_completed,placement_status,company_name,placement_package,interview_rounds_cleared,placement_date
0,1,4001,60,1,Not Ready,None,None,1,None
1,2,4002,87,2,Not Ready,None,None,2,None
2,3,4003,72,1,Ready,None,None,5,None
3,4,4004,88,1,Placed,"Barber, Lee and Johnson",12.12,1,2024-08-01
4,5,4005,63,3,Placed,"Fox, Villa and Mcmillan",24.93,4,2024-10-04


In [64]:
%%writefile app.py

import streamlit as st
import mysql.connector
import pandas as pd
import config

# Establish database connection
def get_db_connection():
    return mysql.connector.connect(
        host=config.DB_HOST,
        user=config.DB_USER,
        password=config.DB_PASSWORD,
        database=config.DB_NAME
    )

# Streamlit UI
st.set_page_config(page_title="Student Placement Filter", layout="wide")

# Sidebar - Page Selection
page = st.sidebar.radio("Select Page", ["Eligibility Filter", "Placement Insights"])

if page == "Eligibility Filter":
    st.title("📌 Student Placement Eligibility Filter")
    
    # Input criteria
    min_soft_skills = st.slider("Minimum Soft Skills Score", min_value=50, max_value=100, value=70)
    min_problems_solved = st.slider("Minimum Problems Solved", min_value=10, max_value=200, value=50)
    
    # Fetch eligible students
    query = f"""
    SELECT s.student_id, s.name, s.age, s.course_batch, p.problems_solved, 
           ROUND((ss.communication + ss.teamwork + ss.presentation + ss.leadership + ss.critical_thinking + ss.interpersonal_skills) / 6, 2) AS avg_soft_skills,
           CASE WHEN p.problems_solved >= {min_problems_solved} 
                AND ROUND((ss.communication + ss.teamwork + ss.presentation + ss.leadership + ss.critical_thinking + ss.interpersonal_skills) / 6, 2) >= {min_soft_skills} 
                THEN 'Eligible' ELSE 'Not Eligible' END AS eligibility_status,
           pl.placement_status
    FROM Students s
    JOIN Programming p ON s.student_id = p.student_id
    JOIN SoftSkills ss ON s.student_id = ss.student_id
    JOIN Placements pl ON s.student_id = pl.student_id
    """
    
    conn = get_db_connection()
    df = pd.read_sql(query, conn)
    conn.close()
    
    st.dataframe(df)

elif page == "Placement Insights":
    st.title("📊 Placement Insights")
    
    queries = {
        "Average Placement Package": "SELECT AVG(placement_package) AS avg_package FROM Placements",
        "Top 5 Students by Mock Interview Score": "SELECT s.name, p.mock_interview_score FROM Students s JOIN Placements p ON s.student_id = p.student_id ORDER BY p.mock_interview_score DESC LIMIT 5",
        "Programming Language Popularity": "SELECT language, COUNT(*) AS student_count FROM Programming GROUP BY language ORDER BY student_count DESC",
        "Soft Skills Performance Distribution": "SELECT AVG(communication) AS avg_comm, AVG(teamwork) AS avg_teamwork, AVG(presentation) AS avg_presentation FROM SoftSkills",
        "Internships vs. Placement": "SELECT internships_completed, COUNT(*) AS student_count FROM Placements GROUP BY internships_completed",
        "Top 5 Students by Problems Solved": "SELECT s.name, p.problems_solved FROM Students s JOIN Programming p ON s.student_id = p.student_id ORDER BY p.problems_solved DESC LIMIT 5",
        "Placement Readiness Ratio": "SELECT placement_status, COUNT(*) AS student_count FROM Placements GROUP BY placement_status",
        "Average Interview Rounds Cleared": "SELECT AVG(interview_rounds_cleared) AS avg_rounds FROM Placements",
        "Most Common Graduation Year Among Students": "SELECT graduation_year, COUNT(*) AS count FROM Students GROUP BY graduation_year ORDER BY count DESC LIMIT 1",
        "Top Companies Hiring": "SELECT company_name, COUNT(*) AS student_count FROM Placements GROUP BY company_name ORDER BY student_count DESC LIMIT 5"
    }
    
    # Dropdown to select a query
    selected_query = st.selectbox("Select an Insight", list(queries.keys()))
    
    # Fetch and display the result for the selected query
    conn = get_db_connection()
    query = queries[selected_query]
    df = pd.read_sql(query, conn)
    conn.close()
    
    # Display the result
    st.subheader(selected_query)
    st.dataframe(df)

    
   
    
   
   
   


    
   


   
       

 
   

        

  

Overwriting app.py


In [68]:
!streamlit run app.py

^C
